In [122]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split

import lightgbm as ltb

import seaborn as sns

In [123]:
train_df = pd.read_csv("../Datasets/new_df.csv")

In [124]:
train_df.dropna()

,Unnamed: 0,id,date,year_week,product_number,prod_category_1,prod_category_2,prod_category_3,prod_category_4,prod_category_5,prod_category_6,prod_category_7,specs,display_size,segment_1,segment_2,segment_3,setmana,sales_units,inventory_units
0,0,202119-6909,2021-05-15,202119,6909,1,0,0,0,0,0,0,1,13.3,1,0,0,19,2.0,35.0
1,1,202120-6909,2021-05-22,202120,6909,1,0,0,0,0,0,0,1,13.3,1,0,0,20,0.0,70.0
2,2,202121-6909,2021-05-29,202121,6909,1,0,0,0,0,0,0,1,13.3,1,0,0,21,3.0,137.0
3,3,202122-6909,2021-06-05,202122,6909,1,0,0,0,0,0,0,1,13.3,1,0,0,22,0.0,274.0
4,4,202123-6909,2021-06-12,202123,6909,1,0,0,0,0,0,0,1,13.3,1,0,0,23,0.0,333.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7704,7704,202314-247737,2023-04-08,202314,247737,1,0,0,0,0,0,0,100,13.3,1,0,0,14,92.0,44.0
7705,7705,202315-247737,2023-04-15,202315,247737,1,0,0,0,0,0,0,100,13.3,1,0,0,15,20.0,30.0
7706,7706,202316-247737,2023-04-22,202316,247737,1,0,0,0,0,0,0,100,13.3,1,0,0,16,12.0,37.0
7707,7707,202317-247737,2023-04-29,202317,247737,1,0,0,0,0,0,0,100,13.3,1,0,0,17,12.0,65.0


In [125]:
train_df['nweek'] = train_df['year_week'].map(lambda x: str(x)[-2:]).astype(np.int64)
train_df['date'] = pd.to_datetime(train_df['date'])
train_df['year'], train_df['month'] = train_df['date'].dt.year, train_df['date'].dt.month
train_df = train_df.drop(['date'],axis=1)
train_df = train_df.drop(['id'],axis=1)

In [126]:
train_2021 = train_df[(train_df['year'] == 2021) | (train_df['year'] == 2020) | (train_df['year'] == 2019)]
train_2022 = train_df[train_df['year'] == 2022]

In [127]:
X = train_2021.drop(columns=['inventory_units'])
Y = train_2021['inventory_units']
X_2022 = train_2022.drop(columns=['inventory_units'])
Y_2022 = train_2022['inventory_units']

In [128]:
X = X.drop(columns = ['Unnamed: 0', 'year_week', 'sales_units'])
X_2022 = X_2022.drop(columns = ['Unnamed: 0', 'year_week', 'sales_units'])

In [129]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle = False)

In [133]:
model = ltb.LGBMRegressor(boosting_type = 'dart',
                          num_leaves = 30,
                          max_depth = 200,
                          learning_rate = 0.15,
                          n_estimators = 50,
                          importance_type  = 'gain',
                          n_jobs = -1)

model.fit(X_train, y_train)

LGBMRegressor(boosting_type='dart', importance_type='gain', learning_rate=0.15,
              max_depth=200, n_estimators=50, num_leaves=30)

In [134]:
from sklearn.metrics import mean_squared_error

y_predicted_train = model.predict(X_train)
rms_train = mean_squared_error(y_train, y_predicted_train, squared=False)
print(f"RMS train: {rms_train}")

y_predicted_test = model.predict(X_test)
rms_test = mean_squared_error(y_test, y_predicted_test, squared=False)
print(f"RMS test: {rms_test}")

RMS train: 42.459893747029426
RMS test: 162.03091977906104


In [135]:
from sklearn.metrics import mean_squared_error

y_predicted_train = model.predict(X_2022)
rms_train = mean_squared_error(Y_2022, y_predicted_train, squared=False)
print(f"RMS 2022: {rms_train}")

RMS 2022: 137.37992871120846
